In [1]:
enum ExprType {
    Application,
    Abstraction,
    Var,
    LiteralInt,
    LiteralBool,
}
type Expr =
    | { type: ExprType.Application, lambda: Expr, argument: Expr }
    | { type: ExprType.Abstraction, binding: Binding, body: Expr }
    | { type: ExprType.Var, name: Ident }
    | { type: ExprType.LiteralInt, value: number }
    | { type: ExprType.LiteralBool, value: boolean }

type Ident = string
type Binding = { name: Ident }

In [ ]:
//#region seen
import { Runtime } from 'jsr:@kawcco/parsebox'
import { OurModule } from "../grammar.ts"

const { Const, Tuple, Union, Ident, Ref, Array, Optional } = Runtime

const Tokens = {
    Arrow: Const('->'),
    LParen: Const('('),
    RParen: Const(')'),
    LBracket: Const('['),
    RBracket: Const(']'),
    Int: Const('int'),
    Bool: Const('bool'),
    Fn: Const('fn'),
    Comma: Const(','),
    Colon: Const(':'),
    Let: Const('let'),
    In: Const('in'),
    Equals: Const('='),
    If: Const('if'),
    Then: Const('then'),
    Else: Const('else'),
    True: Const('true'),
    False: Const('false'),
}
//#endregion

const Language = new OurModule({
    Expr: Tuple(
        [
            Ref<Expr>('ExprWithoutApplication'),
            Array(Tuple([
                Tokens.LParen,
                Ref<Expr>('Expr'),
                Tokens.RParen,
            ], ([, expr,]) => expr)),
        ],
        ([base, applicationArgs]) => {
            let expr = base
            for (const argument of applicationArgs) {
                expr = { type: ExprType.Application, lambda: expr, argument }
            }
            return expr
        }
    ),
    ExprWithoutApplication: Union(
        [
            Ref<Expr>('Abstraction'),
            Ref<Expr>('Int'),
            Ref<Expr>('Bool'),
            Ref<Expr>('Var'),
            Ref<Expr>('ExprParen'),
        ]
    ),
    ExprParen: Tuple(
        [
            Tokens.LParen,
            Ref<Expr>('Expr'),
            Tokens.RParen,
        ],
        ([, expr,]) => expr
    ),
    /// @impl
    Binding: Tuple(
        __________
    ),
    Abstraction: Tuple(
        [
            Ref<Binding>('Binding'),
            Tokens.Arrow,
            Ref<Expr>('Expr')
        ],
        ([binding, , body]) => ({ type: ExprType.Abstraction, binding, body } satisfies Expr),
    ),
    Var: Tuple(
        [
            Ident(),
        ],
        ([name]) => ({ type: ExprType.Var, name } satisfies Expr),
    ),
    //#region seen
    Digit: Union([
        Const("0"),
        Const("1"),
        Const("2"),
        Const("3"),
        Const("4"),
        Const("5"),
        Const("6"),
        Const("7"),
        Const("8"),
        Const("9"),
    ]),
    Int: Tuple(
        [
            Optional(Const("-")),
            Ref("Digit"),
            Array(Ref("Digit"))
        ],
        ([[minus], first_digit, digits]) => (
            {
                type: ExprType.LiteralInt,
                value: parseInt((minus ?? "") + first_digit + digits.join(""))
            } satisfies Expr
        ),
    ),
    Bool: Union(
        [
            Tokens.True,
            Tokens.False,
        ],
        raw => ({ type: ExprType.LiteralBool, value: raw == "true" } satisfies Expr),
    ),
    //#endregion
})


In [2]:
enum ValueType {
    Int,
    Bool,
    Function,
    FunctionNative,
}
type Value =
    | { type: ValueType.Int, value: number }
    | { type: ValueType.Bool, value: boolean }
    | { type: ValueType.Function, body: Expr, binding: Ident, context: Context }

type Context = Record<Ident, Value>


function evaluate(expr: Expr, context: Context = {}): Value {
    switch (expr.type) {
        case ExprType.Application: {
            const lambda = evaluate(expr.lambda, context)
            const argument = evaluate(expr.argument, context)
            if (lambda.type === ValueType.Function) {
                return evaluate(lambda.body, { ...lambda.context, [lambda.binding]: argument })
            }
            throw new Error("Runtime eval error: cannot call type " + ValueType[lambda.type])
        }
        case ExprType.Abstraction: {
            const binding = expr.binding.name
            const body = expr.body
            return {
                type: ValueType.Function,
                binding,
                body,
                context,
            }
        }
        case ExprType.Var: {
            const value = context[expr.name]
            if (value == undefined) {
                throw new Error("Runtime eval error: variable `" + expr.name + "` not found")
            }
            return value
        }
        //#region seen
        case ExprType.LiteralInt: {
            return { type: ValueType.Int, value: expr.value }
        }
        case ExprType.LiteralBool: {
            return { type: ValueType.Bool, value: expr.value }
        }
        //#endregion
    }
}


In [6]:
//////////////////////////////////// REPL TIME ///////////////////////////////////////
import { repl } from "../repl.ts"
await repl(null, Language, evaluate as never)